In [1]:
import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from arch import arch_model
import yfinance as yf
from statsmodels.tsa.stattools import adfuller

def prepare_data(file_path=None, ticker='^GSPC', start_date='2000-01-01', end_date='2023-12-31'):
    if file_path:
        df = pd.read_csv(file_path, parse_dates=['Date'], index_col='Date')
    else:
        df = yf.download(ticker, start=start_date, end=end_date)
        df.to_csv(f'{ticker}.csv')  # Save the data to a CSV file for future use

    df['returns'] = df['Adj Close'].pct_change().dropna() *100 # Use adjusted close prices
    df['close'] = df['Adj Close']
    return df.dropna()

# Function to calculate rolling variance (7-day, 30-day, 60-day)
def calculate_rolling_variance(df):
    df['7d_var'] = df['returns'].rolling(window=7).var() * 252  # Annualize the variance
    df['30d_var'] = df['returns'].rolling(window=30).var() * 252
    df['60d_var'] = df['returns'].rolling(window=60).var() * 252
    return df

# Function to split data into time periods: Pre-COVID, In-COVID, and Post-COVID
def split_time_periods(df):
    # Define date ranges for different periods
    pre_covid_end = '2020-01-01'
    in_covid_end = '2021-04-30'
    
    # Split the data into different time periods
    pre_covid = df[:pre_covid_end]
    in_covid = df[pre_covid_end:in_covid_end]
    post_covid = df[in_covid_end:]
    
    return pre_covid, in_covid, post_covid


def fit_various_garch_models(returns, test_percentage=0.3):
    # Calculate split point
    split_point = int(len(returns) * (1 - test_percentage))
    
    # Split data into train and test sets
    returns_train = returns[:split_point]
    returns_test = returns[split_point:]
    
    models = {
        'GARCH(1,1)': {
            'vol': 'GARCH', 
            'p': 1, 
            'q': 1,
            'power': 2  # Default for standard GARCH
        },
        'EGARCH(1,1)': {
            'vol': 'EGARCH', 
            'p': 1, 
            'q': 1,
            'power': 2  # Default for EGARCH
        },
        'TGARCH(1,1)': {
            'vol': 'GARCH', 
            'p': 1, 
            'q': 1, 
            'power': 1,  # Asymmetric impact
            'o': 1  # Threshold term
        },
        'GJR-GARCH(1,1)': {
            'vol': 'GARCH', 
            'p': 1, 
            'q': 1, 
            'o': 1,  # Asymmetry term
            'power': 2
        },
    }
    
    results = {}
    forecast_volatilities = {}
    fitted_volatilities = {}
    metrics_df = pd.DataFrame()
    
    # Wrap models with tqdm for progress tracking
    model_items = tqdm.tqdm(models.items(), desc="Fitting GARCH Models", total=len(models))
    
    for model_name, model_params in model_items:
        try:
            # Dynamically create the model with specified parameters
            model = arch_model(
                returns_train, 
                vol=model_params['vol'], 
                p=model_params.get('p', 1), 
                q=model_params.get('q', 1),
                o=model_params.get('o', 0),
                power=model_params.get('power', 2)
            )
            
            # Fit the model
            result = model.fit(disp='off', update_freq=0, show_warning=False)
            
            # Store model evaluation metrics
            metrics = {
                'Model': model_name,
                'AIC': result.aic,
                'BIC': result.bic,
                'Log Likelihood': result.loglikelihood,
            }
            
            metrics_df = pd.concat([metrics_df, pd.DataFrame([metrics])], ignore_index=True)
            
            # In-sample fitted volatilities
            fitted_volatilities[model_name] = np.sqrt(result.conditional_volatility.values * 252)
            
            # Rolling forecast on test set
            rolling_forecast_volatilities = []
            rolling_forecast_errors = []
            
            # Wrap test set iteration with tqdm
            test_iterator = tqdm.tqdm(
                range(len(returns_test)), 
                desc=f"Rolling Forecast - {model_name}", 
                leave=False
            )
            
            # We'll do a rolling window forecast
            for i in test_iterator:
                # Prepare rolling training window
                rolling_train = returns[:split_point + i]
                
                # Fit model on rolling training window
                rolling_model = arch_model(
                    rolling_train, 
                    vol=model_params['vol'], 
                    p=model_params.get('p', 1), 
                    q=model_params.get('q', 1),
                    o=model_params.get('o', 0),
                    power=model_params.get('power', 2)
                )
                rolling_result = rolling_model.fit(disp='off', update_freq=0, show_warning=False)
                
                # Forecast next point
                forecast = rolling_result.forecast(horizon=1, start=None, reindex=False, method='bootstrap')
                forecast_variance = forecast.variance.iloc[-1, :].values * 252
                forecast_volatility = np.sqrt(forecast_variance)
                
                rolling_forecast_volatilities.append(forecast_volatility[0])
                
                # Calculate forecast error for this point
                actual_volatility = returns_test.values[i]
                forecast_error = (actual_volatility - forecast_volatility[0]) ** 2
                rolling_forecast_errors.append(forecast_error)
            
            # Store forecast volatilities
            forecast_volatilities[model_name] = np.array(rolling_forecast_volatilities)
            
            # Calculate various loss metrics for the test set
            mse_test = np.mean(rolling_forecast_errors)
            mae_test = np.mean(np.abs(returns_test.values - np.array(rolling_forecast_volatilities)))
            
            # Add loss metrics to the dataframe
            metrics_df.loc[metrics_df['Model'] == model_name, 'MSE_Test'] = mse_test
            metrics_df.loc[metrics_df['Model'] == model_name, 'MAE_Test'] = mae_test
            
            results[model_name] = result
        
        except Exception as e:
            print(f"Error fitting {model_name}: {e}")
            continue
    
    # Sort metrics by MSE to help identify the best performing model
    metrics_df = metrics_df.sort_values('MSE_Test')
    
    return forecast_volatilities, fitted_volatilities, results, metrics_df

In [2]:
# Main execution
file_path = None  # Change to None to fetch data from yfinance
df = prepare_data(file_path=file_path)

# Calculate rolling variance (7-day, 30-day, 60-day)
df = calculate_rolling_variance(df)

# Split data into pre-COVID, in-COVID, and post-COVID periods
pre_covid, in_covid, post_covid = split_time_periods(df)

# For each time period, fit the models and plot the results
time_periods = {'Pre-COVID': pre_covid, 'In-COVID': in_covid, 'Post-COVID': post_covid}
metrics_df = None
for period_name, period_df in time_periods.items():
    print(f"\nFitting models for {period_name} period...")
    
    # Use returns column for model fitting
    returns = period_df['returns']
    
    # Fit the models and get the forecasts
    forecasted_volatilities, fitted_volatilities, results, metrics_df = fit_various_garch_models(returns)
    
    # Print metrics
    print(f"\n{period_name} Model Performance Metrics:")

    #Metrics saved in corresponding csvs
    metrics_df.to_csv(f'{period_name}_model_performance_metrics.csv', index=False)
    
    # Display model parameters
    print(f"\n{period_name} Model Parameters:")

    #Last fitted model based on the rolling window's results displayed here
    for model_name, result in results.items():
        print(f"\n{model_name} Parameters:")
        print(result.summary())

[*********************100%***********************]  1 of 1 completed
C:\Users\Dhruv\AppData\Local\Temp\ipykernel_22136\3122746032.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['7d_var'] = df['returns'].rolling(window=7).var() * 252  # Annualize the variance
C:\Users\Dhruv\AppData\Local\Temp\ipykernel_22136\3122746032.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['30d_var'] = df['returns'].rolling(window=30).var() * 252
C:\Users\Dhruv\AppData\Local\Temp\ipykernel_22136\3122746032.py:24: S


Fitting models for Pre-COVID period...


Fitting GARCH Models: 100%|██████████| 4/4 [02:59<00:00, 44.83s/it]



Pre-COVID Model Performance Metrics:

Pre-COVID Model Parameters:

GARCH(1,1) Parameters:
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -5142.94
Distribution:                  Normal   AIC:                           10293.9
Method:            Maximum Likelihood   BIC:                           10318.5
                                        No. Observations:                 3521
Date:                Wed, Nov 27 2024   Df Residuals:                     3520
Time:                        23:10:48   Df Model:                            1
                                 Mean Model                                 
                 coef    std err          t      P>|t|      95.0% Conf. Int.
--------------------------------------------

Fitting GARCH Models: 100%|██████████| 4/4 [00:06<00:00,  1.64s/it]



In-COVID Model Performance Metrics:

In-COVID Model Parameters:

GARCH(1,1) Parameters:
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -430.674
Distribution:                  Normal   AIC:                           869.349
Method:            Maximum Likelihood   BIC:                           883.170
                                        No. Observations:                  234
Date:                Wed, Nov 27 2024   Df Residuals:                      233
Time:                        23:13:48   Df Model:                            1
                                Mean Model                                
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------

Fitting GARCH Models: 100%|██████████| 4/4 [00:14<00:00,  3.58s/it]


Post-COVID Model Performance Metrics:

Post-COVID Model Parameters:

GARCH(1,1) Parameters:
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -731.839
Distribution:                  Normal   AIC:                           1471.68
Method:            Maximum Likelihood   BIC:                           1488.29
                                        No. Observations:                  470
Date:                Wed, Nov 27 2024   Df Residuals:                      469
Time:                        23:13:54   Df Model:                            1
                                 Mean Model                                
                 coef    std err          t      P>|t|     95.0% Conf. Int.
--------------------------------------------

In [3]:
metrics_df

,Model,AIC,BIC,Log Likelihood,MSE_Test,MAE_Test
3,"GJR-GARCH(1,1)",1456.988968,1477.752632,-723.494484,163.361300,12.514426
2,"TGARCH(1,1)",1451.598186,1472.361850,-720.799093,178.061442,12.994906
0,"GARCH(1,1)",1471.677430,1488.288361,-731.838715,180.032819,13.268962
1,"EGARCH(1,1)",1471.339671,1487.950602,-731.669836,189.574158,13.606578
